In [60]:
# Dependencies
import requests
import time
import csv
import pandas as pd
import datetime as dt
import calendar

In [63]:
#importing the list of areas and zips
zips = pd.read_csv("Resources/zips_ca.csv")
zips.loc[zips["area"]=="bakersfield"]
locations = zips["area"].unique()
locations

array(['bakersfield', 'chico', 'fresno', 'goldcountry', 'hanford',
       'humboldt', 'imperial', 'inlandempire', 'losangeles', 'mendocino',
       'merced', 'modesto', 'monterey', 'orangecounty', 'palmsprings',
       'redding', 'sacramento', 'sandiego', 'santabarbara', 'santamaria',
       'sfbay', 'siskiyou', 'slo', 'stockton', 'susanville', 'ventura',
       'visalia', 'yubasutter'], dtype=object)

In [5]:
###creating a dictionary of the areas and their zips

area_zips = {}

for area in zips["area"].unique():
    local_zip = zips.loc[zips["area"]==area]
    local_zip = local_zip["zip"]
    
    for zip_ in local_zip:
        if area not in area_zips.keys():
            area_zips[area]=[zip_]
        else:
            area_zips[area].append(zip_)
    
area_zips["bakersfield"]

[93203,
 93205,
 93206,
 93207,
 93215,
 93220,
 93222,
 93224,
 93225,
 93226,
 93238,
 93240,
 93241,
 93243,
 93250,
 93251,
 93255,
 93260,
 93263,
 93268,
 93276,
 93283,
 93285,
 93287,
 93301,
 93304,
 93305,
 93306,
 93307,
 93308,
 93309,
 93311,
 93312,
 93313,
 93314,
 93501,
 93505,
 93518,
 93519,
 93527,
 93528,
 93531,
 93554,
 93555,
 93558,
 93561,
 93562,
 93592]

In [291]:
area_zips["redding"]

[96001,
 96002,
 96003,
 96007,
 96008,
 96011,
 96013,
 96016,
 96019,
 96022,
 96024,
 96028,
 96029,
 96033,
 96047,
 96048,
 96051,
 96052,
 96059,
 96062,
 96065,
 96069,
 96073,
 96074,
 96075,
 96076,
 96080,
 96084,
 96087,
 96088,
 96091,
 96093,
 96096]

In [6]:
###creating a dictionary of the categories and their ids from ebay

cat_ids = {"antiques": [20081],
           "arts_crafts": [14339],
           "atv_utv_sno": [43962, 66466],
           "auto_parts": [6030],
           "aviation": [63676],
           "baby_kids": [2984, 3082, 171146],
           "beauty_health": [26395],
           "bike_parts": [7294],
           "bikes": [177831],
           "boat_parts": [26443],
           "boats": [26429],
           "books": [267],
           "cars_trucks": [6001],
           "cds_dvds_vhs": [176984, 617, 309],
           "cell_phones": [15032],
           "clothes_acc": [15724, 1059, 14324, 3034, 93427, 169291, 4251, 4250],
           "computer_parts": [175673],
           "computers": [58058],
           "furniture": [3197],
           "heavy_equip": [257887],
           "jewelry": [4196, 10968, 48579, 91427, 10290],
           "motorcycle_parts": [10063],
           "motorcycles": [6024],
           "music_intr": [619],
           "photo_video": [625],
           "rvs_camp": [50054],
           "sporting": [888],
           "tools": [631],
           "toys_games": [220],
           "trailers": [66468],
           "video_gaming": [1249]
          }

for k, v in cat_ids.items():
    print(k)
    for id in v:
        print(id)

antiques
20081
arts_crafts
14339
atv_utv_sno
43962
66466
auto_parts
6030
aviation
63676
baby_kids
2984
3082
171146
beauty_health
26395
bike_parts
7294
bikes
177831
boat_parts
26443
boats
26429
books
267
cars_trucks
6001
cds_dvds_vhs
176984
617
309
cell_phones
15032
clothes_acc
15724
1059
14324
3034
93427
169291
4251
4250
computer_parts
175673
computers
58058
furniture
3197
heavy_equip
257887
jewelry
4196
10968
48579
91427
10290
motorcycle_parts
10063
motorcycles
6024
music_intr
619
photo_video
625
rvs_camp
50054
sporting
888
tools
631
toys_games
220
trailers
66468
video_gaming
1249


In [9]:
###generating the dataframe of urls to run a json query from

#first key is Kyle's, second is Sean's
api_key = ["KyleDavi-DAProjec-PRD-5721a6b08-1eb0707a", "SeanTibb-SeanTibb-PRD-b7da08f45-1e07c1f5"]

url_df = []
max_api_call_per_day_count = 0
api_alternate = 0
loc_url_count = 0

url_df = pd.DataFrame(columns=["location", "zip", "category", "cat_id", "url"])

for loc, zips in area_zips.items():
    loc_url_count = 0
    print("Location: "+ loc + " started.")  #if errors, to check where
    for zip_ in zips:
        for cat, ids in cat_ids.items():
            id_urls = {}
            for id in ids:
                loc_url_count +=1
                max_api_call_per_day_count +=1
                if max_api_call_per_day_count > 5000: #max calls per key per day is 5000
                    api_alternate += 1 #used to alternate between api keys.  we will modulus to swap later
                    max_api_call_per_day_count = 0 #resets if we're reaching 5000 calls
                
                query_url = "https://svcs.ebay.com/services/search/FindingService/v1?\
OPERATION-NAME=findItemsByCategory&\
SERVICE-VERSION=1.0.0&\
SECURITY-APPNAME="+api_key[api_alternate%2]+"&\
buyerPostalCode="+str(zip_)+"&\
itemFilter.name=MaxDistance&\
itemFilter.value=5&\
RESPONSE-DATA-FORMAT=JSON&\
REST-PAYLOAD&\
categoryId="+str(id)+"&\
paginationInput.entriesPerPage=1000"
            
                temp_df = {"location":loc, 
                           "zip":zip_, 
                           "category":cat, 
                           "cat_id":id, 
                           "url":query_url}
                
                url_df = url_df.append(temp_df, ignore_index=True)
    print("Location: "+ loc + " finished, "+str(loc_url_count)+" total urls generated.")  #if errors, to check where
    print("-------------------------------------")

url_df.head()

Location: bakersfield started.
Location: bakersfield finished, 2256 total urls generated.
-------------------------------------
Location: chico started.
Location: chico finished, 1222 total urls generated.
-------------------------------------
Location: fresno started.
Location: fresno finished, 1786 total urls generated.
-------------------------------------
Location: goldcountry started.
Location: goldcountry finished, 2961 total urls generated.
-------------------------------------
Location: hanford started.
Location: hanford finished, 846 total urls generated.
-------------------------------------
Location: humboldt started.
Location: humboldt finished, 2068 total urls generated.
-------------------------------------
Location: imperial started.
Location: imperial finished, 846 total urls generated.
-------------------------------------
Location: inlandempire started.
Location: inlandempire finished, 1175 total urls generated.
-------------------------------------
Location: losangel

,location,zip,category,cat_id,url
0,bakersfield,93203,antiques,20081,https://svcs.ebay.com/services/search/FindingS...
1,bakersfield,93203,arts_crafts,14339,https://svcs.ebay.com/services/search/FindingS...
2,bakersfield,93203,atv_utv_sno,43962,https://svcs.ebay.com/services/search/FindingS...
3,bakersfield,93203,atv_utv_sno,66466,https://svcs.ebay.com/services/search/FindingS...
4,bakersfield,93203,auto_parts,6030,https://svcs.ebay.com/services/search/FindingS...


In [40]:
#url_df.to_csv("Resources/full_list_of_urls.csv", index=False)

In [76]:
imported_df = []
filtered_url_df = []

imported_df = pd.read_csv("Resources/full_list_of_urls.csv")
bakersfield_url_df = imported_df.loc[imported_df["location"]=="bakersfield"]
sacramento_url_df = imported_df.loc[imported_df["location"]=="sacramento"]
redding_url_df = imported_df.loc[imported_df["location"]=="redding"]
sandiego_url_df = imported_df.loc[imported_df["location"]=="sandiego"]

In [73]:
len(redding_url_df), len(bakersfield_url_df), len(sacramento_url_df), len(sandiego_url_df)

(1551, 2256, 3525, 4606)

In [81]:
filtered_url_df = []
filtered_url_df = pd.DataFrame
filtered_url_df = pd.concat([bakersfield_url_df, redding_url_df, sacramento_url_df, sandiego_url_df])
filtered_url_df = filtered_url_df.reset_index(drop=True)
filtered_url_df.to_csv("Resources/filtered_list_of_urls.csv", index=False)

In [82]:
filtered_url_df

,location,zip,category,cat_id,url
0,bakersfield,93203,antiques,20081,https://svcs.ebay.com/services/search/FindingS...
1,bakersfield,93203,arts_crafts,14339,https://svcs.ebay.com/services/search/FindingS...
2,bakersfield,93203,atv_utv_sno,43962,https://svcs.ebay.com/services/search/FindingS...
3,bakersfield,93203,atv_utv_sno,66466,https://svcs.ebay.com/services/search/FindingS...
4,bakersfield,93203,auto_parts,6030,https://svcs.ebay.com/services/search/FindingS...
5,bakersfield,93203,aviation,63676,https://svcs.ebay.com/services/search/FindingS...
6,bakersfield,93203,baby_kids,2984,https://svcs.ebay.com/services/search/FindingS...
7,bakersfield,93203,baby_kids,3082,https://svcs.ebay.com/services/search/FindingS...
8,bakersfield,93203,baby_kids,171146,https://svcs.ebay.com/services/search/FindingS...
9,bakersfield,93203,beauty_health,26395,https://svcs.ebay.com/services/search/FindingS...


# code from here on out will need to be babysat

In [286]:
###change for each day.  "first_day...", "second_day...", etc
###be sure to save into new csv
todays_requests = pd.read_csv("Resources/first_day_of_calls_urls.csv")
todays_requests = import_requests.iloc[0:8526]
todays_requests

,location,zip,category,cat_id,url
0,bakersfield,93203,antiques,20081,https://svcs.ebay.com/services/search/FindingS...
1,bakersfield,93203,arts_crafts,14339,https://svcs.ebay.com/services/search/FindingS...
2,bakersfield,93203,atv_utv_sno,43962,https://svcs.ebay.com/services/search/FindingS...
3,bakersfield,93203,atv_utv_sno,66466,https://svcs.ebay.com/services/search/FindingS...
4,bakersfield,93203,auto_parts,6030,https://svcs.ebay.com/services/search/FindingS...
5,bakersfield,93203,aviation,63676,https://svcs.ebay.com/services/search/FindingS...
6,bakersfield,93203,baby_kids,2984,https://svcs.ebay.com/services/search/FindingS...
7,bakersfield,93203,baby_kids,3082,https://svcs.ebay.com/services/search/FindingS...
8,bakersfield,93203,baby_kids,171146,https://svcs.ebay.com/services/search/FindingS...
9,bakersfield,93203,beauty_health,26395,https://svcs.ebay.com/services/search/FindingS...


In [280]:
todays_requests = todays_requests[0:10]

In [287]:
###
json_list = []
json_count = 0
for index, row in todays_requests.iterrows():
    json_count+= 1
    this_request = requests.get(row["url"]).json()
    json_list.append([row["url"], this_request])
    print("Pulling request "+str(json_count))
        
json_list[0]

Pulling request 1
Pulling request 2
Pulling request 3
Pulling request 4
Pulling request 5
Pulling request 6
Pulling request 7
Pulling request 8
Pulling request 9
Pulling request 10
Pulling request 11
Pulling request 12
Pulling request 13
Pulling request 14
Pulling request 15
Pulling request 16
Pulling request 17
Pulling request 18
Pulling request 19
Pulling request 20
Pulling request 21
Pulling request 22
Pulling request 23
Pulling request 24
Pulling request 25
Pulling request 26
Pulling request 27
Pulling request 28
Pulling request 29
Pulling request 30
Pulling request 31
Pulling request 32
Pulling request 33
Pulling request 34
Pulling request 35
Pulling request 36
Pulling request 37
Pulling request 38
Pulling request 39
Pulling request 40
Pulling request 41
Pulling request 42
Pulling request 43
Pulling request 44
Pulling request 45
Pulling request 46
Pulling request 47
Pulling request 48
Pulling request 49
Pulling request 50
Pulling request 51
Pulling request 52
Pulling request 53
Pu

Pulling request 417
Pulling request 418
Pulling request 419
Pulling request 420
Pulling request 421
Pulling request 422
Pulling request 423
Pulling request 424
Pulling request 425
Pulling request 426
Pulling request 427
Pulling request 428
Pulling request 429
Pulling request 430
Pulling request 431
Pulling request 432
Pulling request 433
Pulling request 434
Pulling request 435
Pulling request 436
Pulling request 437
Pulling request 438
Pulling request 439
Pulling request 440
Pulling request 441
Pulling request 442
Pulling request 443
Pulling request 444
Pulling request 445
Pulling request 446
Pulling request 447
Pulling request 448
Pulling request 449
Pulling request 450
Pulling request 451
Pulling request 452
Pulling request 453
Pulling request 454
Pulling request 455
Pulling request 456
Pulling request 457
Pulling request 458
Pulling request 459
Pulling request 460
Pulling request 461
Pulling request 462
Pulling request 463
Pulling request 464
Pulling request 465
Pulling request 466


Pulling request 827
Pulling request 828
Pulling request 829
Pulling request 830
Pulling request 831
Pulling request 832
Pulling request 833
Pulling request 834
Pulling request 835
Pulling request 836
Pulling request 837
Pulling request 838
Pulling request 839
Pulling request 840
Pulling request 841
Pulling request 842
Pulling request 843
Pulling request 844
Pulling request 845
Pulling request 846
Pulling request 847
Pulling request 848
Pulling request 849
Pulling request 850
Pulling request 851
Pulling request 852
Pulling request 853
Pulling request 854
Pulling request 855
Pulling request 856
Pulling request 857
Pulling request 858
Pulling request 859
Pulling request 860
Pulling request 861
Pulling request 862
Pulling request 863
Pulling request 864
Pulling request 865
Pulling request 866
Pulling request 867
Pulling request 868
Pulling request 869
Pulling request 870
Pulling request 871
Pulling request 872
Pulling request 873
Pulling request 874
Pulling request 875
Pulling request 876


Pulling request 1226
Pulling request 1227
Pulling request 1228
Pulling request 1229
Pulling request 1230
Pulling request 1231
Pulling request 1232
Pulling request 1233
Pulling request 1234
Pulling request 1235
Pulling request 1236
Pulling request 1237
Pulling request 1238
Pulling request 1239
Pulling request 1240
Pulling request 1241
Pulling request 1242
Pulling request 1243
Pulling request 1244
Pulling request 1245
Pulling request 1246
Pulling request 1247
Pulling request 1248
Pulling request 1249
Pulling request 1250
Pulling request 1251
Pulling request 1252
Pulling request 1253
Pulling request 1254
Pulling request 1255
Pulling request 1256
Pulling request 1257
Pulling request 1258
Pulling request 1259
Pulling request 1260
Pulling request 1261
Pulling request 1262
Pulling request 1263
Pulling request 1264
Pulling request 1265
Pulling request 1266
Pulling request 1267
Pulling request 1268
Pulling request 1269
Pulling request 1270
Pulling request 1271
Pulling request 1272
Pulling reque

Pulling request 1617
Pulling request 1618
Pulling request 1619
Pulling request 1620
Pulling request 1621
Pulling request 1622
Pulling request 1623
Pulling request 1624
Pulling request 1625
Pulling request 1626
Pulling request 1627
Pulling request 1628
Pulling request 1629
Pulling request 1630
Pulling request 1631
Pulling request 1632
Pulling request 1633
Pulling request 1634
Pulling request 1635
Pulling request 1636
Pulling request 1637
Pulling request 1638
Pulling request 1639
Pulling request 1640
Pulling request 1641
Pulling request 1642
Pulling request 1643
Pulling request 1644
Pulling request 1645
Pulling request 1646
Pulling request 1647
Pulling request 1648
Pulling request 1649
Pulling request 1650
Pulling request 1651
Pulling request 1652
Pulling request 1653
Pulling request 1654
Pulling request 1655
Pulling request 1656
Pulling request 1657
Pulling request 1658
Pulling request 1659
Pulling request 1660
Pulling request 1661
Pulling request 1662
Pulling request 1663
Pulling reque

...

Pulling request 7091
Pulling request 7092
Pulling request 7093
Pulling request 7094
Pulling request 7095
Pulling request 7096
Pulling request 7097
Pulling request 7098
Pulling request 7099
Pulling request 7100
Pulling request 7101
Pulling request 7102
Pulling request 7103
Pulling request 7104
Pulling request 7105
Pulling request 7106
Pulling request 7107
Pulling request 7108
Pulling request 7109
Pulling request 7110
Pulling request 7111
Pulling request 7112
Pulling request 7113
Pulling request 7114
Pulling request 7115
Pulling request 7116
Pulling request 7117
Pulling request 7118
Pulling request 7119
Pulling request 7120
Pulling request 7121
Pulling request 7122
Pulling request 7123
Pulling request 7124
Pulling request 7125
Pulling request 7126
Pulling request 7127
Pulling request 7128
Pulling request 7129
Pulling request 7130
Pulling request 7131
Pulling request 7132
Pulling request 7133
Pulling request 7134
Pulling request 7135
Pulling request 7136
Pulling request 7137
Pulling reque

Pulling request 7482
Pulling request 7483
Pulling request 7484
Pulling request 7485
Pulling request 7486
Pulling request 7487
Pulling request 7488
Pulling request 7489
Pulling request 7490
Pulling request 7491
Pulling request 7492
Pulling request 7493
Pulling request 7494
Pulling request 7495
Pulling request 7496
Pulling request 7497
Pulling request 7498
Pulling request 7499
Pulling request 7500
Pulling request 7501
Pulling request 7502
Pulling request 7503
Pulling request 7504
Pulling request 7505
Pulling request 7506
Pulling request 7507
Pulling request 7508
Pulling request 7509
Pulling request 7510
Pulling request 7511
Pulling request 7512
Pulling request 7513
Pulling request 7514
Pulling request 7515
Pulling request 7516
Pulling request 7517
Pulling request 7518
Pulling request 7519
Pulling request 7520
Pulling request 7521
Pulling request 7522
Pulling request 7523
Pulling request 7524
Pulling request 7525
Pulling request 7526
Pulling request 7527
Pulling request 7528
Pulling reque

Pulling request 7873
Pulling request 7874
Pulling request 7875
Pulling request 7876
Pulling request 7877
Pulling request 7878
Pulling request 7879
Pulling request 7880
Pulling request 7881
Pulling request 7882
Pulling request 7883
Pulling request 7884
Pulling request 7885
Pulling request 7886
Pulling request 7887
Pulling request 7888
Pulling request 7889
Pulling request 7890
Pulling request 7891
Pulling request 7892
Pulling request 7893
Pulling request 7894
Pulling request 7895
Pulling request 7896
Pulling request 7897
Pulling request 7898
Pulling request 7899
Pulling request 7900
Pulling request 7901
Pulling request 7902
Pulling request 7903
Pulling request 7904
Pulling request 7905
Pulling request 7906
Pulling request 7907
Pulling request 7908
Pulling request 7909
Pulling request 7910
Pulling request 7911
Pulling request 7912
Pulling request 7913
Pulling request 7914
Pulling request 7915
Pulling request 7916
Pulling request 7917
Pulling request 7918
Pulling request 7919
Pulling reque

Pulling request 8264
Pulling request 8265
Pulling request 8266
Pulling request 8267
Pulling request 8268
Pulling request 8269
Pulling request 8270
Pulling request 8271
Pulling request 8272
Pulling request 8273
Pulling request 8274
Pulling request 8275
Pulling request 8276
Pulling request 8277
Pulling request 8278
Pulling request 8279
Pulling request 8280
Pulling request 8281
Pulling request 8282
Pulling request 8283
Pulling request 8284
Pulling request 8285
Pulling request 8286
Pulling request 8287
Pulling request 8288
Pulling request 8289
Pulling request 8290
Pulling request 8291
Pulling request 8292
Pulling request 8293
Pulling request 8294
Pulling request 8295
Pulling request 8296
Pulling request 8297
Pulling request 8298
Pulling request 8299
Pulling request 8300
Pulling request 8301
Pulling request 8302
Pulling request 8303
Pulling request 8304
Pulling request 8305
Pulling request 8306
Pulling request 8307
Pulling request 8308
Pulling request 8309
Pulling request 8310
Pulling reque

['https://svcs.ebay.com/services/search/FindingService/v1?OPERATION-NAME=findItemsByCategory&SERVICE-VERSION=1.0.0&SECURITY-APPNAME=KyleDavi-DAProjec-PRD-5721a6b08-1eb0707a&buyerPostalCode=93203&itemFilter.name=MaxDistance&itemFilter.value=5&RESPONSE-DATA-FORMAT=JSON&REST-PAYLOAD&categoryId=20081&paginationInput.entriesPerPage=1000',
 {'findItemsByCategoryResponse': [{'ack': ['Success'],
    'version': ['1.13.0'],
    'timestamp': ['2019-07-04T07:02:56.612Z'],
    'searchResult': [{'@count': '0'}],
    'paginationOutput': [{'pageNumber': ['0'],
      'entriesPerPage': ['100'],
      'totalPages': ['0'],
      'totalEntries': ['0']}],
    'itemSearchURL': ['http://www.ebay.com/sch/20081/i.html?LH_Distance=93203..5&_ddo=1&_fpos=93203&_fspt=1&_ipg=100&_pgn=1&_pos=93203&_sadis=5&_stpos=93203']}]}]

In [295]:
json_list[10]#["findItemsByCategoryResponse"][0]["searchResult"][0]["item"]#[1]["shippingInfo"][0]["shippingServiceCost"]

['https://svcs.ebay.com/services/search/FindingService/v1?OPERATION-NAME=findItemsByCategory&SERVICE-VERSION=1.0.0&SECURITY-APPNAME=KyleDavi-DAProjec-PRD-5721a6b08-1eb0707a&buyerPostalCode=93203&itemFilter.name=MaxDistance&itemFilter.value=5&RESPONSE-DATA-FORMAT=JSON&REST-PAYLOAD&categoryId=7294&paginationInput.entriesPerPage=1000',
 {'findItemsByCategoryResponse': [{'ack': ['Success'],
    'version': ['1.13.0'],
    'timestamp': ['2019-07-04T07:03:01.550Z'],
    'searchResult': [{'@count': '10',
      'item': [{'itemId': ['153552509987'],
        'title': ['Powerlite expert bmx frame cruiser early 2000'],
        'globalId': ['EBAY-US'],
        'primaryCategory': [{'categoryId': ['22679'],
          'categoryName': ['Bicycle Frames']}],
        'galleryURL': ['http://thumbs4.ebaystatic.com/m/mkKB2amRqY_mng1FwzmZOXQ/140.jpg'],
        'viewItemURL': ['http://www.ebay.com/itm/Powerlite-expert-bmx-frame-cruiser-early-2000-/153552509987'],
        'paymentMethod': ['PayPal'],
        'au

In [98]:
json_list_temp_df = pd.DataFrame(json_list)

# json_list_temp_df.to_csv("Resources/temp_json_file.csv", index=False)

In [267]:
json_list[1][0]

'https://svcs.ebay.com/services/search/FindingService/v1?OPERATION-NAME=findItemsByCategory&SERVICE-VERSION=1.0.0&SECURITY-APPNAME=KyleDavi-DAProjec-PRD-5721a6b08-1eb0707a&buyerPostalCode=92007&itemFilter.name=MaxDistance&itemFilter.value=5&RESPONSE-DATA-FORMAT=JSON&REST-PAYLOAD&categoryId=15724&paginationInput.entriesPerPage=1000'

In [ ]:
##testing how to pull json elements out
json_df_one = []
json_df_two = []
json_df_three = []
json_df_four = []

json_df_one = pd.DataFrame(columns=["url", "results_count", "result_title", "result_currency",\
                               "result_list_price", "result_ship_price", "free_shipping"])
json_df_two = pd.DataFrame(columns=["url", "results_count", "result_title", "result_currency",\
                               "result_list_price", "result_ship_price", "free_shipping"])
json_df_three = pd.DataFrame(columns=["url", "results_count", "result_title", "result_currency",\
                               "result_list_price", "result_ship_price", "free_shipping"])
json_df_four = pd.DataFrame(columns=["url", "results_count", "result_title", "result_currency",\
                               "result_list_price", "result_ship_price", "free_shipping"])

#
# if (json_list.index(this_thing)//2500) == 0:
#     #append to df1
# elif (json_list.index(this_thing)//2500) == 1:
#     #append to df2
# elif (json_list.index(this_thing)//2500) == 2:
#     #append to df3
# else:
#     #append to df4

for this_thing in json_list:
    results_count = 0
    result_title = ""
    result_currency = ""
    result_list_price = 0
    result_ship_price = 0
    free_shipping = ""
    try:
        this_search = this_thing[1]["findItemsByCategoryResponse"][0]["searchResult"][0]
    ###number of results
        results_count = this_search["@count"]
        if int(results_count) > 0:
            for the_item in this_search["item"]:

            ###title
                result_title = this_search["item"][this_search["item"].index(the_item)]['title']
            ###currency
                result_currency = this_search["item"][this_search["item"].index(the_item)]["sellingStatus"][0]["currentPrice"][0]["@currencyId"]
            ###price
                result_list_price = this_search["item"][this_search["item"].index(the_item)]["sellingStatus"][0]["currentPrice"][0]["__value__"]
            ###shipping price
                try:
                    result_ship_price = this_search["item"][this_search["item"].index(the_item)]["shippingInfo"][0]["shippingServiceCost"][0]["__value__"]
                ###free shipping
                    free_shipping = this_search["item"][this_search["item"].index(the_item)]["shippingInfo"][0]["shippingType"][0] == 'Free'

                except:
                    result_ship_price = "unknown"
                    free_shipping = False

                temp_df = {"url": this_thing[0],
                      "results_count":results_count,
                      "result_title": result_title, 
                      "result_currency": result_currency,
                      "result_list_price": result_list_price,
                      "result_ship_price": result_ship_price,
                      "free_shipping": free_shipping}
                if (json_list.index(this_thing)//2500) == 0:
                    #append to df1
                    json_df_one = json_df_one.append(temp_df, ignore_index=True)
                elif (json_list.index(this_thing)//2500) == 1:
                    #append to df2
                    json_df_two = json_df_two.append(temp_df, ignore_index=True)
                elif (json_list.index(this_thing)//2500) == 2:
                    #append to df3
                    json_df_three = json_df_three.append(temp_df, ignore_index=True)
                else:
                    #append to df4
                    json_df_four = json_df_four.append(temp_df, ignore_index=True)

        else:
            temp_df = {"url": this_thing[0],
                      "results_count":results_count,
                      "result_title": result_title, 
                      "result_currency": result_currency,
                      "result_list_price": result_list_price,
                      "result_ship_price": result_ship_price,
                      "free_shipping": free_shipping}

            if (json_list.index(this_thing)//2500) == 0:
                #append to df1
                json_df_one = json_df_one.append(temp_df, ignore_index=True)
            elif (json_list.index(this_thing)//2500) == 1:
                #append to df2
                json_df_two = json_df_two.append(temp_df, ignore_index=True)
            elif (json_list.index(this_thing)//2500) == 2:
                #append to df3
                json_df_three = json_df_three.append(temp_df, ignore_index=True)
            else:
                #append to df4
                json_df_four = json_df_four.append(temp_df, ignore_index=True)  

        print(json_list.index(this_thing))
    except:
            temp_df = {"url": this_thing[0],
                      "results_count": results_count,
                      "result_title": "ERROR", 
                      "result_currency": result_currency,
                      "result_list_price": result_list_price,
                      "result_ship_price": result_ship_price,
                      "free_shipping": free_shipping}
            if (json_list.index(this_thing)//2500) == 0:
                #append to df1
                json_df_one = json_df_one.append(temp_df, ignore_index=True)
            elif (json_list.index(this_thing)//2500) == 1:
                #append to df2
                json_df_two = json_df_two.append(temp_df, ignore_index=True)
            elif (json_list.index(this_thing)//2500) == 2:
                #append to df3
                json_df_three = json_df_three.append(temp_df, ignore_index=True)
            else:
                #append to df4
                json_df_four = json_df_four.append(temp_df, ignore_index=True)  

In [289]:
json_df = pd.concat([json_df_one, json_df_two, json_df_three, json_df_four], ignore_index=True)
# len(json_df_one)+len(json_df_two)
json_df["url"][200]

'https://svcs.ebay.com/services/search/FindingService/v1?OPERATION-NAME=findItemsByCategory&SERVICE-VERSION=1.0.0&SECURITY-APPNAME=KyleDavi-DAProjec-PRD-5721a6b08-1eb0707a&buyerPostalCode=93203&itemFilter.name=MaxDistance&itemFilter.value=5&RESPONSE-DATA-FORMAT=JSON&REST-PAYLOAD&categoryId=10968&paginationInput.entriesPerPage=1000'

In [270]:
json_df["url"][500]

'https://svcs.ebay.com/services/search/FindingService/v1?OPERATION-NAME=findItemsByCategory&SERVICE-VERSION=1.0.0&SECURITY-APPNAME=KyleDavi-DAProjec-PRD-5721a6b08-1eb0707a&buyerPostalCode=92007&itemFilter.name=MaxDistance&itemFilter.value=5&RESPONSE-DATA-FORMAT=JSON&REST-PAYLOAD&categoryId=93427&paginationInput.entriesPerPage=1000'

In [290]:
###change file name and 
json_df.to_csv("Resources/zero_day_json_parsed_FIXED.csv", index=False)

In [ ]:
### place to assign that day's pulls
day_one_json_pulls = json_list

In [194]:
json_df = pd.concat([json_df_one, json_df_two, json_df_three, json_df_four], ignore_index=True)
# len(json_df_one)+len(json_df_two)

In [88]:
test_list = [[1,2], [3,4]]
test_list

[[1, 2], [3, 4]]

In [66]:
test_list.append(1)

In [67]:
test_list

[1]

In [93]:
for item in test_list:
    print(test_list.index(item))

0
1


In [180]:
(4999//2500)

1

In [169]:
json_list[1][1]

{'findItemsByCategoryResponse': [{'ack': ['Success'],
   'version': ['1.13.0'],
   'timestamp': ['2019-06-29T21:57:10.658Z'],
   'searchResult': [{'@count': '13',
     'item': [{'itemId': ['362507715486'],
       'title': ['Handmade Paper Beads Glazed Recycled Loose Painted Lot 50 Ct - PPM01'],
       'globalId': ['EBAY-US'],
       'primaryCategory': [{'categoryId': ['146290'],
         'categoryName': ['Beads']}],
       'galleryURL': ['http://thumbs3.ebaystatic.com/m/mE7msvkGZbOybSuQFrEoG3w/140.jpg'],
       'viewItemURL': ['http://www.ebay.com/itm/Handmade-Paper-Beads-Glazed-Recycled-Loose-Painted-Lot-50-Ct-PPM01-/362507715486'],
       'paymentMethod': ['PayPal'],
       'autoPay': ['false'],
       'postalCode': ['93203'],
       'location': ['Arvin,CA,USA'],
       'country': ['US'],
       'shippingInfo': [{'shippingServiceCost': [{'@currencyId': 'USD',
           '__value__': '0.0'}],
         'shippingType': ['Free'],
         'shipToLocations': ['Worldwide'],
         'exped

In [240]:
this_thing[0]

'https://svcs.ebay.com/services/search/FindingService/v1?OPERATION-NAME=findItemsByCategory&SERVICE-VERSION=1.0.0&SECURITY-APPNAME=SeanTibb-SeanTibb-PRD-b7da08f45-1e07c1f5&buyerPostalCode=93624&itemFilter.name=MaxDistance&itemFilter.value=5&RESPONSE-DATA-FORMAT=JSON&REST-PAYLOAD&categoryId=3082&paginationInput.entriesPerPage=1000'